In [ ]:
import os
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import torch
from dysts.base import DynSys
from dysts.metrics import compute_metrics
from scipy.integrate import solve_ivp
from tqdm import tqdm, trange

from panda.chronos.pipeline import ChronosPipeline
from panda.patchtst.pipeline import PatchTSTPipeline
from panda.utils.data_utils import safe_standardize
from panda.utils.plot_utils import apply_custom_style

device_rank = 3

apply_custom_style("../config/plotting.yaml")

In [ ]:
figs_save_dir = "../figures/kuramoto_sivashinsky"
os.makedirs(figs_save_dir, exist_ok=True)

## KS Equation

In [ ]:
class KuramotoShivashinsky(DynSys):
    """Implements the 1+1D KS equation in fourier space"""

    def __init__(self, L: float, modes: int):
        self.ic = None
        super().__init__(metadata_path=None, dimension=2 * modes, parameters={})
        self.L = L
        self.modes = modes
        self.dimension = 2 * self.modes
        self.wave_nums = 2 * np.pi * np.arange(0, self.modes + 2) / self.L
        self.N = self.dimension + 2

        # precompute some quantities
        self.freq_domain = np.zeros(self.modes + 2, dtype=np.complex128)
        self.nonlinear_factor = -0.5 * 1j * self.wave_nums * self.N
        self.diffusion_ffts = self.wave_nums**2 - self.wave_nums**4

    def to_spatial(self, q: np.ndarray, N: int) -> np.ndarray:
        """Inverse FFT of the modes to get u(x) at a certain time

        :param q: array of flattened fourier coefficients (real and imag components), can have batch dimensions
        :param N: grid resolution in the spatial domain

        :returns: solution in the spatial domain
        """
        coeffs = np.zeros(q.shape[:-1] + (self.modes + 2,), dtype=complex)
        coeffs[..., 1:-1] = q[..., : self.modes] + 1j * q[..., self.modes :]
        return np.fft.irfft(coeffs, n=N)

    def rhs(self, t: float, X: np.ndarray) -> np.ndarray:
        self.freq_domain[1:-1] = X[: self.modes] + 1j * X[self.modes :]
        u = np.fft.irfft(self.freq_domain, n=self.N)
        pseudospectral_term = self.nonlinear_factor * np.fft.rfft(u * u)
        linear_term = self.diffusion_ffts * self.freq_domain

        # repackage components
        flow = (linear_term + pseudospectral_term)[1:-1]
        return np.concatenate([np.real(flow), np.imag(flow)])

In [ ]:
ks = KuramotoShivashinsky(L=100, modes=64)

tfinal = 100
rng = np.random.default_rng(12)  # 1234
ic = 0.1 * rng.normal(size=(ks.dimension,))
teval = np.linspace(0, tfinal, 4096)
sol = solve_ivp(ks.rhs, (0, tfinal), ic, method="DOP853", t_eval=teval, rtol=1e-8, atol=1e-8)
ts, freq_traj = sol.t, sol.y.T
spatial_traj = ks.to_spatial(freq_traj, N=ks.dimension)

In [ ]:
grid = np.linspace(0, ks.L, ks.dimension)
plt.figure(figsize=(10, 4))
plt.pcolormesh(ts, grid, spatial_traj.T, cmap="RdBu", shading="gouraud")
plt.colorbar()
plt.ylabel("x")
plt.xlabel("t")
plt.show()

In [ ]:
run_name = "pft_chattn_emb_w_poly-0"
pipeline = PatchTSTPipeline.from_pretrained(
    mode="predict",
    pretrain_path=f"/stor/work/AMDG_Gilpin_Summer2024/checkpoints/{run_name}/checkpoint-final",
    device_map=f"cuda:{device_rank}",
    torch_dtype=torch.float32,
)
pipeline

# Forecast Visualization

In [ ]:
def forecast(
    pipeline,
    trajectory: np.ndarray,
    context_length: int,
    normalize: bool = True,
    transpose: bool = False,
    prediction_length: int | None = None,
    **kwargs,
) -> np.ndarray:
    context = trajectory[:context_length]
    if normalize:
        context = safe_standardize(context, axis=0)

    if prediction_length is None:
        prediction_length = trajectory.shape[0] - context_length

    if transpose:
        context = context.T

    predictions = (
        pipeline.predict(
            context=torch.tensor(context).float(),
            prediction_length=prediction_length,
            limit_prediction_length=False,
            **kwargs,
        )
        .squeeze()
        .cpu()
        .numpy()
    )
    full_trajectory = np.concatenate([context, predictions], axis=1 if transpose else 0)

    if transpose:
        full_trajectory = full_trajectory.T

    if normalize:
        return safe_standardize(
            full_trajectory,
            axis=0,
            context=trajectory[:context_length],
            denormalize=True,
        )

    return full_trajectory

In [ ]:
def plot_forecast(
    ts: np.ndarray,
    grid: np.ndarray,
    trajectory: np.ndarray,
    predictions: np.ndarray,
    run_name: str = "",
    context_length: int = 512,
    save_path: str | None = None,
    v_abs: float | None = None,
    prediction_horizon: int = 128,
    cmap_name: str = "RdBu",
    title_kwargs: dict = {},
    draw_prediction_horizon_line: bool = True,
    figsize: tuple[int, int] = (9, 9),
    show_cbar: bool = True,
    show_ticks: bool = True,
    show_axes_labels: bool = True,
    show_context: bool = True,
):
    fig, axes = plt.subplots(3, 1, sharex=True, figsize=figsize)

    vmin = min(trajectory.min(), predictions.min())
    vmax = max(trajectory.max(), predictions.max())
    vabs = v_abs or max(abs(vmin), abs(vmax))

    if not show_context:
        ts = ts[context_length:]

    for i, (ax, data, label) in enumerate(
        zip(
            axes,
            [trajectory, predictions, predictions - trajectory],
            [
                "Ground Truth",
                f"Predictions ({run_name})" if run_name else "Predictions",
                "Error",
                # f"Prediction Error ({np.mean(np.abs(predictions[context_length:] - trajectory[context_length:])):.2e}) ({run_name})",
            ],
        )
    ):
        if not show_context:
            data = data[context_length:]
        im = ax.pcolormesh(ts, grid, data.T, cmap=cmap_name, shading="gouraud", vmin=-vabs, vmax=vabs)
        if show_ticks:
            ax.set_ylabel("x")
        else:
            ax.set_yticks([])

        ax.set_title(label, **title_kwargs)
        if show_cbar:
            fig.colorbar(im, ax=ax)
        # draw black vertical line at middle of plot (x axis middle)

        if show_context:
            ax.axvline(ts[context_length], color="black", linewidth=1)

        if i == 2 and draw_prediction_horizon_line:
            # draw a black dotted vertical line at the end of 128 pred length window
            start = context_length if show_context else 0
            ax.axvline(
                ts[start + prediction_horizon],
                color="gray",
                linestyle="--",
                linewidth=1,
            )
    if show_ticks:
        axes[-1].set_xlabel("t")
        # axes[-1].set_xticks(ts[::256])
    else:
        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])

    if show_axes_labels:
        axes[-1].set_xlabel("Time", fontweight="bold")
        for ax in axes:
            ax.set_ylabel("x", fontweight="bold")
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")

    return vabs

In [ ]:
len(ts)

In [ ]:
start_time = 1024
context_length = 512

prediction_length = 256
end_time = start_time + (context_length + prediction_length)

In [ ]:
use_deterministic = True  # for Chronos

### Our Model

In [ ]:
# # predict in frequency domain
# preds_freq = forecast(
#     pipeline,
#     freq_traj[start_time:end_time],
#     context_length,
#     prediction_length=512,
#     normalize=True,
#     sliding_context=True,
# )

# # convert to spatial domain
# preds_freq_to_spatial = ks.to_spatial(preds_freq, N=ks.dimension)

In [ ]:
# our_freq_vabs = plot_forecast(
#     ts[start_time:end_time],
#     grid,
#     spatial_traj[start_time:end_time],
#     preds_freq_to_spatial,
#     run_name="Panda",
#     context_length=context_length,
#     save_path="../figures/ks_our_model_freq_to_spatial.pdf",
#     cmap_name="RdBu",
# )

In [ ]:
# predict in spatial domain
preds_spatial = forecast(
    pipeline,
    spatial_traj[start_time:end_time],
    context_length,
    prediction_length=prediction_length,
    normalize=True,
    sliding_context=True,
)

In [ ]:
spatial_traj.shape

In [ ]:
ts[end_time]

In [ ]:
figs_save_dir

In [ ]:
our_spatial_vabs = plot_forecast(
    ts[start_time:end_time],
    grid,
    spatial_traj[start_time:end_time],
    preds_spatial,
    # run_name="Panda",
    context_length=context_length,
    save_path=os.path.join(figs_save_dir, "ks_our_model_spatial.pdf"),
    cmap_name="RdBu",
    title_kwargs={"fontweight": "bold", "fontsize": 8},
    show_ticks=True,
    show_axes_labels=True,
    show_cbar=False,
    figsize=(3.2, 5),
    show_context=True,
)

### Chronos Finetune

In [ ]:
chronos_ft = ChronosPipeline.from_pretrained(
    # "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_finetune_stand_updated-0/checkpoint-final",
    # "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_mini_ft-0/checkpoint-final",
    "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_t5_mini_ft-0/checkpoint-final",
    device_map=f"cuda:{device_rank}",
    torch_dtype=torch.float32,
)
chronos_ft

In [ ]:
def forecast_chronos(
    pipeline,
    trajectory: np.ndarray,
    context_length: int,
    chunk_size: int,
    **kwargs,
) -> np.ndarray:
    subchannel_predictions = []
    for i in trange(0, trajectory.shape[1] // chunk_size):
        subpreds = forecast(
            pipeline,
            trajectory[:, i * chunk_size : (i + 1) * chunk_size],
            context_length,
            prediction_length=None,
            transpose=True,
            normalize=False,
            num_samples=1,
            **kwargs,
        )
        subchannel_predictions.append(subpreds)

    return np.concatenate(subchannel_predictions, axis=1)

In [ ]:
# # predict in frequency domain
# chronos_preds_freq = forecast_chronos(
#     chronos_ft, freq_traj[start_time:end_time], context_length, chunk_size=ks.dimension
# )

# # convert to spatial domain
# chronos_preds_freq_to_spatial = ks.to_spatial(chronos_preds_freq, N=ks.dimension)

In [ ]:
# plot_forecast(
#     ts[start_time:end_time],
#     grid,
#     spatial_traj[start_time:end_time],
#     chronos_preds_freq_to_spatial,
#     run_name="Chronos 20M SFT",
#     context_length=context_length,
#     save_path="../figures/ks_chronos_ft_freq_to_spatial.pdf",
#     v_abs=our_freq_vabs,
# )

In [ ]:
# spatial domain chronos prediction
chronos_preds_spatial = forecast_chronos(
    chronos_ft,
    spatial_traj[start_time:end_time],
    context_length,
    chunk_size=ks.dimension,
    deterministic=use_deterministic,
)

In [ ]:
plot_forecast(
    ts[start_time:end_time],
    grid,
    spatial_traj[start_time:end_time],
    chronos_preds_spatial,
    run_name="Chronos 20M SFT",
    context_length=context_length,
    save_path=os.path.join(figs_save_dir, "ks_chronos_ft_spatial.pdf"),
    v_abs=our_spatial_vabs,
    cmap_name="RdBu",
    title_kwargs={"fontweight": "bold", "fontsize": 8},
    show_ticks=False,
    show_cbar=False,
    figsize=(3, 5),
    show_context=True,
)

### Chronos Zeroshot

In [ ]:
chronos_zs = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-mini",
    device_map=f"cuda:{device_rank}",
    torch_dtype=torch.float32,
)
chronos_zs

In [ ]:
# chronos_zs_preds_freq = forecast_chronos(
#     chronos_zs, freq_traj[start_time:end_time], context_length, chunk_size=ks.dimension
# )

# # convert to spatial domain
# chronos_zs_preds_freq_to_spatial = ks.to_spatial(chronos_zs_preds_freq, N=ks.dimension)

In [ ]:
# plot_forecast(
#     ts[start_time:end_time],
#     grid,
#     spatial_traj[start_time:end_time],
#     chronos_zs_preds_freq_to_spatial,
#     run_name="Chronos 20M",
#     context_length=context_length,
#     save_path="../figures/ks_chronos_zs_freq_to_spatial.pdf",
#     v_abs=our_freq_vabs,
# )

In [ ]:
# spatial domain chronos prediction
chronos_zs_preds_spatial = forecast_chronos(
    chronos_zs,
    spatial_traj[start_time:end_time],
    context_length,
    chunk_size=ks.dimension,
    deterministic=use_deterministic,
)

In [ ]:
plot_forecast(
    ts[start_time:end_time],
    grid,
    spatial_traj[start_time:end_time],
    chronos_zs_preds_spatial,
    run_name="Chronos 20M",
    context_length=context_length,
    save_path=os.path.join(figs_save_dir, "ks_chronos_zs_spatial.pdf"),
    v_abs=our_spatial_vabs,
    cmap_name="RdBu",
    title_kwargs={"fontweight": "bold", "fontsize": 8},
    show_ticks=False,
    show_cbar=False,
    figsize=(3, 5),
    show_context=True,
)

# Rollout Evaluation

In [ ]:
# repeated for convenience
start_time = 1024
end_time = 2048
context_length = 512

In [ ]:
n_runs = 40
parent_rng = np.random.default_rng(12)  # 12
rng_stream = parent_rng.spawn(n_runs)

predict_spatial = True  # predict in spatial domain instead of frequency domain
convert_to_spatial = False  # if prediction in freq domain, convert to spatial domain

In [ ]:
trajectories = []

for rng in rng_stream:
    ic = 0.1 * rng.normal(size=(ks.dimension,))
    teval = np.linspace(0, tfinal, 4096)
    sol = solve_ivp(ks.rhs, (0, tfinal), ic, method="DOP853", t_eval=teval, rtol=1e-8, atol=1e-8)
    ts, freq_traj = sol.t, sol.y.T
    if predict_spatial:
        trajectories.append(ks.to_spatial(freq_traj, N=ks.dimension))
    else:
        trajectories.append(freq_traj)

In [ ]:
time_intervals = [(0, end) for end in np.arange(64, 512 + 64, 64)]

In [ ]:
save_dir = "../outputs/kuramoto_sivashinsky"
os.makedirs(save_dir, exist_ok=True)

np.save(os.path.join(save_dir, f"trajectories_{n_runs}.npy"), trajectories)

In [ ]:
# trajectories = np.load(os.path.join(save_dir, f"trajectories_{n_runs}.npy"))

In [ ]:
def compute_pred_error(prediction, ground_truth, time_intervals: list[tuple[int, int]]):
    pred_error_dict = {}
    for start, end in time_intervals:
        error_dict = compute_metrics(
            prediction[start:end],
            ground_truth[start:end],
            include=["mae", "mse", "smape"],
        )
        pred_error_dict[start, end] = error_dict
    return pred_error_dict


def get_mean_median_std_metrics_dicts_rollout(
    predictions: list[np.ndarray],
    trajectories: list[np.ndarray],
    time_intervals: list[tuple[int, int]],
):
    pred_error_dict_lst = []
    for preds, traj in zip(predictions, trajectories):
        actual_preds = preds[context_length:]
        actual_gt = traj[start_time:end_time][context_length:]
        pred_error_dict_lst.append(compute_pred_error(actual_preds, actual_gt, time_intervals))

    metrics_lst = ["mse", "mae", "smape"]
    metric_dict = defaultdict(dict)
    for time_interval in pred_error_dict_lst[0].keys():
        for metric in metrics_lst:
            values = []
            for pred_error_dict in pred_error_dict_lst:
                values.append(pred_error_dict[time_interval][metric])
            values = np.array(values)
            mean_metric = np.mean(values, axis=0)
            median_metric = np.median(values, axis=0)
            std_metric = np.std(values, axis=0)
            metric_dict[time_interval][metric] = {
                "mean": mean_metric,
                "median": median_metric,
                "std": std_metric,
            }

    mean_metrics_dict = defaultdict(dict)
    for time_interval in time_intervals:
        for metric in metrics_lst:
            mean_metrics_dict[metric][time_interval] = metric_dict[time_interval][metric]["mean"]

    median_metrics_dict = defaultdict(dict)
    for time_interval in time_intervals:
        for metric in metrics_lst:
            median_metrics_dict[metric][time_interval] = metric_dict[time_interval][metric]["median"]

    std_metrics_dict = defaultdict(dict)
    for time_interval in time_intervals:
        for metric in metrics_lst:
            std_metrics_dict[metric][time_interval] = metric_dict[time_interval][metric]["std"]

    return mean_metrics_dict, median_metrics_dict, std_metrics_dict

### Our Model

In [ ]:
preds = []

for traj in tqdm(trajectories):
    sample_pred = forecast(
        pipeline,
        traj[start_time:end_time],
        context_length,
        prediction_length=None,
        normalize=True,
        sliding_context=True,
    )
    if convert_to_spatial and not predict_spatial:
        sample_pred = ks.to_spatial(sample_pred, N=ks.dimension)
    preds.append(sample_pred)

In [ ]:
np.save(os.path.join(save_dir, f"our_model_{run_name}_preds_{n_runs}.npy"), preds)

### Chronos Finetune

In [ ]:
use_deterministic = True  # for Chronos

In [ ]:
chronos_ft_preds = []

for traj in tqdm(trajectories):
    chronos_ft_sample_pred = forecast_chronos(
        chronos_ft,
        traj[start_time:end_time],
        context_length,
        chunk_size=ks.dimension,
        deterministic=use_deterministic,
    )
    if convert_to_spatial and not predict_spatial:
        chronos_ft_sample_pred = ks.to_spatial(chronos_ft_sample_pred, N=ks.dimension)
    chronos_ft_preds.append(chronos_ft_sample_pred)

In [ ]:
np.save(
    os.path.join(
        save_dir,
        f"chronos_ft_preds_{n_runs}{'' if use_deterministic else '_nondeterministic'}.npy",
    ),
    chronos_ft_preds,
)

### Chronos Zeroshot

In [ ]:
chronos_zs_preds = []

for traj in tqdm(trajectories):
    chronos_zs_sample_pred = forecast_chronos(
        chronos_zs,
        traj[start_time:end_time],
        context_length,
        chunk_size=ks.dimension,
        deterministic=use_deterministic,
    )
    if convert_to_spatial and not predict_spatial:
        chronos_zs_sample_pred = ks.to_spatial(chronos_zs_sample_pred, N=ks.dimension)
    chronos_zs_preds.append(chronos_zs_sample_pred)

In [ ]:
np.save(
    os.path.join(
        save_dir,
        f"chronos_zs_preds_{n_runs}{'' if use_deterministic else '_nondeterministic'}.npy",
    ),
    chronos_zs_preds,
)

### Plot Results

In [ ]:
# # Load predictions from saved files
# chronos_ft_preds = np.load(
#     os.path.join(
#         save_dir,
#         f"chronos_ft_preds_{n_runs}{'' if use_deterministic else '_nondeterministic'}.npy",
#     )
# )
# chronos_zs_preds = np.load(
#     os.path.join(
#         save_dir,
#         f"chronos_zs_preds_{n_runs}{'' if use_deterministic else '_nondeterministic'}.npy",
#     )
# )
# preds = np.load(os.path.join(save_dir, f"our_model_{run_name}_preds_{n_runs}.npy"))

In [ ]:
markers = ["o", "s", "v", "D", "X"]

In [ ]:
end_times = [end_time for _, end_time in time_intervals]
for metric_to_plot, title_metric_name in [("smape", "sMAPE")]:
    plt.figure(figsize=(4, 3))
    for i, (run_name, plist) in enumerate(
        zip(
            ["Panda", "Chronos 20M SFT", "Chronos 20M"],
            [preds, chronos_ft_preds, chronos_zs_preds],
        )
    ):
        mean_metrics_dict, median_metrics_dict, std_metrics_dict = get_mean_median_std_metrics_dicts_rollout(
            plist, trajectories, time_intervals
        )
        mean_vals = np.array(list(mean_metrics_dict[metric_to_plot].values()))
        std_vals = np.array(list(std_metrics_dict[metric_to_plot].values()))
        plt.plot(
            end_times,
            list(mean_vals),
            label=run_name,
            marker=markers[i],
        )
        plt.fill_between(
            end_times,
            np.array(list(mean_vals)) - np.array(list(std_vals)) / np.sqrt(len(time_intervals)),
            np.array(list(mean_vals)) + np.array(list(std_vals)) / np.sqrt(len(time_intervals)),
            alpha=0.2,
        )
    plt.xticks(end_times)
    plt.legend(loc="lower right", frameon=True)
    # plt.title(f"{title_metric_name}", fontweight="bold")
    plt.title("Kuramoto Sivashinsky", fontweight="bold")
    plt.ylabel(f"{title_metric_name}", fontweight="bold")
    plt.xlabel("Prediction Length", fontweight="bold")
    plt.tight_layout()
    plt.savefig(
        os.path.join(figs_save_dir, f"ks_all_models_{metric_to_plot}.pdf"),
        bbox_inches="tight",
    )
    plt.show()
    plt.close()

In [ ]:
end_times = [end_time for _, end_time in time_intervals]
for metric_to_plot, title_metric_name in [("mae", "MAE")]:
    plt.figure(figsize=(4, 3))
    for i, (run_name, plist) in enumerate(
        zip(
            ["Panda", "Chronos 20M SFT", "Chronos 20M"],
            [preds, chronos_ft_preds, chronos_zs_preds],
        )
    ):
        mean_metrics_dict, median_metrics_dict, std_metrics_dict = get_mean_median_std_metrics_dicts_rollout(
            plist, trajectories, time_intervals
        )
        mean_vals = np.array(list(mean_metrics_dict[metric_to_plot].values()))
        std_vals = np.array(list(std_metrics_dict[metric_to_plot].values()))
        plt.plot(
            end_times,
            list(mean_vals),
            label=run_name,
            marker=markers[i],
        )
        plt.fill_between(
            end_times,
            np.array(list(mean_vals)) - np.array(list(std_vals)) / np.sqrt(len(time_intervals)),
            np.array(list(mean_vals)) + np.array(list(std_vals)) / np.sqrt(len(time_intervals)),
            alpha=0.2,
        )
    plt.xticks(end_times)
    plt.legend(loc="lower right", frameon=True)
    # plt.title(f"{title_metric_name}", fontweight="bold")
    plt.title("Kuramoto Sivashinsky", fontweight="bold")
    plt.ylabel(f"{title_metric_name}", fontweight="bold")
    plt.xlabel("Prediction Length", fontweight="bold")
    # scientific notation
    plt.ticklabel_format(style="sci", axis="y", scilimits=(0, 0))
    plt.tight_layout()
    plt.savefig(
        os.path.join(figs_save_dir, f"ks_all_models_{metric_to_plot}.pdf"),
        bbox_inches="tight",
    )
    plt.show()
    plt.close()